In [10]:
import pandas as pd
import numpy as np
adv_stats = pd.read_csv("adv_stats.csv")
hist_odds = pd.read_csv("hist_odds.csv")
mom_stats = pd.read_csv("momentum.csv")
today_lines = pd.read_csv("today_lines.csv")
team_id = {'Washington' : 1610612764, 'GoldenState' : 1610612744, 'Minnesota' : 1610612750, 'Houston' : 1610612745, 'Milwaukee' : 1610612749, 'Chicago' : 1610612741, 'Philadelphia': 1610612755, 'Memphis' : 1610612763, 'Brooklyn' : 1610612751, 
           'OklahomaCity' : 1610612760, 'Sacramento' : 1610612758, 'Charlotte' : 1610612766, 'Indiana' : 1610612754, 'Toronto': 1610612761, 'SanAntonio' : 1610612759, 'Orlando' : 1610612753, 'Utah' : 1610612762, 'NewOrleans' : 1610612740, 'LALakers': 1610612747, 
           'Portland' : 1610612757, 'Dallas' : 1610612742, 'Atlanta' : 1610612737, 'Boston' : 1610612738, 'Cleveland' : 1610612739, 'Detroit': 1610612765, 'Miami' : 1610612748, 'LAClippers' : 1610612746, 'Denver' : 1610612743, 'Phoenix' : 1610612756, 'NewYork' : 1610612752}
hist_odds = hist_odds.replace(team_id)
hist_odds['win'] = [1 if x > y else 0 for x,y in zip(hist_odds['Final_H'], hist_odds['Final_V'])]
hist_odds['fav'] = [1 if x > y else 0 for x,y in zip(hist_odds['ML_H'], hist_odds['ML_V'])]
home_team = hist_odds[["VH_y", "Team_H", "Final_H"]]
away_team = hist_odds[['VH_x', "Team_V", "Final_V"]]

home_team = pd.merge(left = home_team, right = adv_stats, left_on = "Team_H", right_on = "TEAM_ID", how = "left")
away_team = pd.merge(left = away_team, right = adv_stats, left_on = "Team_V", right_on = "TEAM_ID", how = "left")
model_stats = pd.merge(left = home_team, right = away_team, left_index = True, right_index = True, how = "outer")
model_stats['win'] = hist_odds['win']
model_stats['fav'] = hist_odds['fav']
model_stats.to_csv('model_stats.csv')

In [11]:
to_keep = ['W_PCT_x', 'W_PCT_y', 'VH_y', 'E_OFF_RATING_x', 'E_OFF_RATING_y', 'E_DEF_RATING_x', 'E_DEF_RATING_y', 'E_NET_RATING_x', 'E_PACE_x', 'E_AST_RATIO_x', 'E_OREB_PCT_x' , 'E_DREB_PCT_x',
           'E_REB_PCT_x', 'E_TM_TOV_PCT_x', 'E_NET_RATING_y' , 'E_PACE_y', 'E_AST_RATIO_y', 'E_OREB_PCT_y' ,'E_DREB_PCT_y', 'E_REB_PCT_y', 'E_TM_TOV_PCT_y', 'MIN_x', 'MIN_y', 'fav']
model_stats_x = model_stats[to_keep]
col_names = ['W_PCT_HOME', 'W_PCT_AWAY', 'home_team', 'E_OFF_RATING_HOME', 'E_OFF_RATING_AWAY', 'E_DEF_RATING_HOME', 'E_DEF_RATING_AWAY', 'E_NET_RATING_HOME', 'E_PACE_HOME', 'E_AST_RATIO_HOME', 'E_OREB_PCT_HOME' , 'E_DREB_PCT_HOME',
           'E_REB_PCT_HOME', 'E_TM_TOV_PCT_HOME', 'E_NET_RATING_AWAY' , 'E_PACE_AWAY', 'E_AST_RATIO_AWAY', 'E_OREB_PCT_AWAY' ,'E_DREB_PCT_AWAY', 'E_REB_PCT_AWAY', 'E_TM_TOV_PCT_AWAY', 'MIN_HOME', 'MIN_AWAY', 'fav']
model_stats_x.columns = col_names

# Random Forest - Home Score Predictions


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_H'], test_size=0.2, random_state=300)
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=200, n_jobs = -1)
rf_random.fit(X_train, y_train);


Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [8]:
home_predictions = rf_random.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Accuracy: 92.1 %.


# Random Forest - Away Score Predictions


In [6]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_V'], test_size=0.2, random_state=300)
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=200, n_jobs = -1)
rf_random.fit(X_train, y_train);

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [7]:
home_predictions = rf_random.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Accuracy: 92.1 %.


#    Linear Regression

In [5]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_H'], test_size=0.2, random_state=400)
reg  = LinearRegression()
fit = reg.fit(X_train, y_train)
home_predictions =  fit.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 91.17 %.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_V'], test_size=0.2, random_state=500)
reg  = LinearRegression()
fit = reg.fit(X_train, y_train)
home_predictions =  fit.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 91.09 %.


# Lasso


In [24]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_H'], test_size=0.2, random_state=600)
lasso = Lasso()
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error')
results = search.fit(X_train, y_train)
home_predictions = results.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18847.247972855508, tolerance: 5.051692647058824
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\

Accuracy: 91.3 %.


In [25]:
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_V'], test_size=0.2, random_state=700)
lasso = Lasso()
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error')
results = search.fit(X_train, y_train)
home_predictions = results.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19645.180517125547, tolerance: 5.388535
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\thyang\An

Accuracy: 92.2 %.


# Ridge


In [22]:
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_H'], test_size=0.2, random_state=800)
ridge = Ridge()
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error')
results = search.fit(X_train, y_train)
home_predictions = results.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20116.446349981205, tolerance: 5.201158823529411
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\

Accuracy: 91.59 %.


In [23]:
X_train, X_test, y_train, y_test = train_test_split(model_stats_x, model_stats['Final_V'], test_size=0.2, random_state=900)
ridge = Ridge()
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error')
results = search.fit(X_train, y_train)
home_predictions = results.best_estimator_.predict(X_test)
home_errors = abs(home_predictions - y_test)
mape = 100 * (home_errors /y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19031.04445212802, tolerance: 4.906129411764706
  positive)
C:\Users\thyang\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\t

Accuracy: 91.77 %.
